In [ ]:
import numpy as np 
import pandas as pd 

from sklearn.ensemble import RandomForestClassifier 
from sklearn.feature_extraction.text import CountVectorizer 

from nltk.corpus import stopwords # для удаления стоп-слов
from bs4 import BeautifulSoup # для парсинга
import re # для работы с регулярными выражениями

Считаем датасеты

header=0 - для того, чтобы указать, что первая строка - название столбцов <br>
delimiter="t" - для того, чтобы указать, что строки разделяются символом табуляции <br>
quoting=3 - для того, чтобы указать, что необходимо игнорировать двойные кавычки (во избежании ошибок)

In [ ]:
train_data = pd.read_csv("../input/word2vec-nlp-tutorial/labeledTrainData.tsv.zip", header=0, delimiter="\t", quoting=3)
test_data = pd.read_csv("../input/word2vec-nlp-tutorial/testData.tsv.zip", header=0, delimiter="\t", quoting=3)

print('Размер тренировочной выборки', train_data.shape)
print('Размер тестовой выборки', test_data.shape)

Просмотрим датасеты

In [ ]:
train_data.head()

In [ ]:
test_data.head()

Просмотрим данные, которые содержатся в столбце review (на примере первой строки)

In [ ]:
print("Содержимое столбца 'review' в первой строке тренировочной выборки\n\n", train_data["review"][0])

Создадим функцию для удаления html кода, знаков препинания, "не важных" слов

In [ ]:
def review_to_words(raw_review):
    rev_text = BeautifulSoup(raw_review).get_text()
    l = re.sub("[^a-zA-Z]", " ", rev_text)
    low_case = l.lower()
    words = low_case.split()
    stop = set(stopwords.words("english"))
    words_n = [word for word in words if not word in stop]
    return( " ".join(words_n))

Займемся препроцессингом данных

In [ ]:
train_data['review']=[review_to_words(review) for review in train_data['review']]
cleaned_train_data = []
for i in range(0, train_data["review"].size):
    cleaned_train_data.append( review_to_words( train_data["review"][i] ))

In [ ]:
cleaned_test_data = []
for i in range(0, len(test_data["review"])):
    cleaned_test_data.append( review_to_words( test_data["review"][i] ))

Посмотрим, как изменились данные 

In [ ]:
print("Измененное содержимое столбца 'review' в первой строке тренировочной выборки\n\n", cleaned_train_data[0])

Сформируем вектор на основании обработанных данных из тренировочной выборки

In [ ]:
vector = CountVectorizer(analyzer = "word", tokenizer = None, preprocessor = None, stop_words = None, max_features = 25000)
x_train = vector.fit_transform(cleaned_train_data)
x_train = x_train.toarray()

print(x_train.shape)

Сформируем вектор на основании обработанных данных из тестовой выборки

In [ ]:
x_test = vector.transform(cleaned_test_data)
x_test = x_test.toarray()

Обучим модель, используя Random Forest Classifier

In [ ]:
y_train = train_data["sentiment"]
model = RandomForestClassifier(n_estimators = 200)
model.fit(x_train, y_train)
print('Точность Random forest:', model.score(x_train, y_train))

Предскажем данные для тестовой выборки

In [ ]:
predicted_data = model.predict(x_test)

Запишем результат

In [ ]:
result = pd.DataFrame(data={"id": test_data["id"], "sentiment": predicted_data})
result.to_csv('prediction.csv', index=False, quoting=3)